In [1]:
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter

### TO DO
- [] Look into concordance analysis
- [] read paper again -- results conclusion?

 STUCK :-(
- [] rewrite stroma calculation within a dataframe (clean up function)
- [X] Calculate allCuts (Youden Index + ROC) --> cf. develop script R
- [] Get results for one stage at a time only --> ask michael
- [X] Write as function

In [88]:
df = pd.read_excel(r"C:\Users\amali\Desktop\TCGA_MEASUREMENTS.xlsx")

x = df.to_numpy()
x[:,-1] = x[:,-1] / 365.35 # convert 'days to event' to 'years to event'
x[:,11] = x[:,11] / 10 # convert 'years to birth' to 'decades to birth'
x = np.append(x, np.zeros((500,1)), axis=1) # add column for HD score

# Calculate allWeights
classes_col = ["ADI","BACK", "DEB", "LYM", "MUC","MUS", "NORM", "STR","TUM"]
df["years_to_event"] = df["days_to_event"]/365.25 # convert 'days to event' to 'years to event'
df["decades_to_birth"] = df["years_to_birth"]/10 # convert 'years to birth' to 'decades to birth'
y = df[["years_to_event","vital_status"]]

cph_models = [CoxPHFitter().fit(pd.concat([df[col], y], axis=1), "years_to_event", "vital_status") for col in classes_col]
allWeights = np.array([float(cph.summary["exp(coef)"]) for cph in cph_models])

# Calculate allCuts?
allCuts = np.array([0.00056, 0.00227, 0.03151, 0.00121, 0.01123, 0.02359, 0.06405, 0.00122, 0.99961]) # Youden cuts

# Calculate stroma score
scoreIndices = (np.argwhere(allWeights >= 1)).flatten()
for i in scoreIndices:
    x[:,-1] = x[:,-1] + (x[:,i+1]>=allCuts[i])*allWeights[i] # +1 retrieve column number in x
medianTrainingSet = np.median(x[:,-1])
x[:,-1] = (x[:,-1] >= medianTrainingSet)*1
stroma_score = x[:,-1]

runs=["All stages", "Stage i", "Stage ii", "Stage iii", "Stage iv"]
df["stroma_score"] = stroma_score

selected_columns = ["stroma_score" ,"cleanstage", "gender", "decades_to_birth" ]
df_mv = df.dropna(subset=["cleanstage", "decades_to_birth"])
y_mv = df_mv[["years_to_event","vital_status"]]

mv_cox = CoxPHFitter().fit(pd.concat([df_mv[selected_columns], y_mv], axis=1), "years_to_event", "vital_status", formula = "stroma_score + cleanstage + C(gender) +decades_to_birth")
# mv_cox.print_summary()

stroma_HR = mv_cox.hazard_ratios_[3]
lci = np.exp(mv_cox.confidence_intervals_["95% lower-bound"][3])
hci = np.exp(mv_cox.confidence_intervals_["95% upper-bound"][3])
p = mv_cox._compute_p_values()[3]
print(f"___Deep Stroma Score Hazard Ratio, {runs[0]}___ \n Stroma HR: {stroma_HR}; \n CI: [{lci}, {hci}]; \n p: {p}")


___Deep Stroma Score Hazard Ratio, All stages___ 
 Stroma HR: 1.9854308859239034; 
 CI: [1.2654457286170446, 3.1150571799618465]; 
 p: 0.002841235882829334


In [89]:
mv_cox.summary

,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,cmp to,z,p,-log2(p)
covariate,,,,,,,,,,,
C(gender)[T.male],-0.320635,0.725688,0.206700,-0.725759,0.084489,0.483957,1.088161,0.0,-1.551212,1.208509e-01,3.048700
cleanstage,0.856957,2.355981,0.119149,0.623430,1.090485,1.865315,2.975716,0.0,7.192325,6.369737e-13,40.513831
decades_to_birth,0.403958,1.497741,0.091499,0.224623,0.583293,1.251850,1.791929,0.0,4.414880,1.010662e-05,16.594340
stroma_score[T.1],0.685836,1.985431,0.229806,0.235424,1.136248,1.265446,3.115057,0.0,2.984412,2.841236e-03,8.459266


In [28]:
mv_cox.concordance_index_

0.7505731069560857

In [4]:
## CALCULATE ALL_WEIGHTS
classes_col = ["ADI","BACK", "DEB", "LYM", "MUC","MUS", "NORM", "STR","TUM"]
df["years_to_event"] = df["days_to_event"]/365.25 # convert 'days to event' to 'years to event'
df["decades_to_birth"] = df["years_to_birth"]/10 # convert 'years to birth' to 'decades to birth'
y = df[["years_to_event","vital_status"]]

cph_models = [CoxPHFitter().fit(pd.concat([df[col], y], axis=1), "years_to_event", "vital_status") for col in classes_col]
allWeights = np.array([float(cph.summary["exp(coef)"]) for cph in cph_models])
allWeights

array([1.14988747, 0.01458188, 5.96672736, 1.22524921, 0.48793896,
       3.7607187 , 0.90871294, 1.15375135, 0.47529061])

In [5]:
## CALCULATE STROMA SCORE
x = df.to_numpy()
# print(x.shape)
x[:,-1] = x[:,-1] / 365.35 # convert 'days to event' to 'years to event'
x[:,11] = x[:,11] / 10 # convert 'years to birth' to 'decades to birth'
x = np.append(x, np.zeros((500,1)), axis=1) # add column for HD score

allCuts = np.array([0.00056, 0.00227, 0.03151, 0.00121, 0.01123, 0.02359, 0.06405, 0.00122, 0.99961]) # Youden cuts
# allWeights = np.array([1.150, 0.015, 5.967, 1.226, 0.488, 3.761, 0.909, 1.154, 0.475]) # HR
# medianTrainingSet = 8.347

scoreIndices = (np.argwhere(allWeights >= 1)).flatten()
for i in scoreIndices:
    x[:,-1] = x[:,-1] + (x[:,i+1]>=allCuts[i])*allWeights[i] # +1 retrieve column number in x
medianTrainingSet = np.median(x[:,-1])
x[:,-1] = (x[:,-1] >= medianTrainingSet)*1

stroma_score = x[:,-1]
stroma_score[0:9]

array([0, 0, 1, 1, 1, 1, 0, 0, 0], dtype=object)

In [76]:
df_mv[selected_columns]

,stroma_score,cleanstage,gender,decades_to_birth
0,0,4.0,1,3.5
1,0,4.0,1,4.4
2,1,4.0,1,4.7
3,1,4.0,1,7.5
4,1,4.0,1,8.0
...,...,...,...,...
479,1,1.0,1,6.1
480,1,1.0,0,5.3
481,0,1.0,0,7.1
482,0,1.0,0,6.3


In [90]:
data_path = r"C:\Users\amali\Desktop\TCGA_MEASUREMENTS.xlsx"
df = pd.read_excel(data_path)
df["years_to_event"] = df["days_to_event"]/365.25 # convert 'days to event' to 'years to event'
df["decades_to_birth"] = df["years_to_birth"]/10 # convert 'years to birth' to 'decades to birth'

runs=["All stages", "Stage i", "Stage ii", "Stage iii", "Stage iv"]
df["stroma_score"] = stroma_score

selected_columns = ["stroma_score" ,"cleanstage", "gender", "decades_to_birth" ]
df_mv = df.dropna(subset=["cleanstage", "decades_to_birth"])
y_mv = df_mv[["years_to_event","vital_status"]]

mv_cox = CoxPHFitter().fit(pd.concat([df_mv[selected_columns], y_mv], axis=1), "years_to_event", "vital_status", formula = "stroma_score + cleanstage + C(gender) +decades_to_birth")
lci = np.exp(mv_cox.confidence_intervals_["95% lower-bound"][3])
hci = np.exp(mv_cox.confidence_intervals_["95% upper-bound"][3])
p = mv_cox._compute_p_values()[3]
print(f"___Deep Stroma Score Hazard Ratio, {runs[0]}___ \n Stroma HR: {stroma_HR}; \n CI: [{lci}, {hci}]; \n p: {p} \n")

y_mvi = y_mv[df_mv["cleanstage"] == 1]
df_mvi = df_mv[df_mv["cleanstage"] == 1]
# mv_coxi = CoxPHFitter().fit(pd.concat([df_mvi[selected_columns], y_mvi], axis=1), "years_to_event", "vital_status", formula = "stroma_score + cleanstage + gender +decades_to_birth")

___Deep Stroma Score Hazard Ratio, All stages___ 
 Stroma HR: 1.9854308859239034; 
 CI: [1.2654457286170446, 3.1150571799618465]; 
 p: 0.002841235882829334 



In [92]:
mv_coxi = CoxPHFitter().fit(pd.concat([df_mvi[selected_columns], y_mvi], axis=1), "years_to_event", "vital_status", formula = "stroma_score + gender +decades_to_birth")

In [93]:
mv_coxi.summary

,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,cmp to,z,p,-log2(p)
covariate,,,,,,,,,,,
decades_to_birth,0.441171,1.554527,0.348742,-0.242350,1.124693,0.784781,3.079270,0.0,1.265037,0.205858,2.280277
gender[T.male],0.079132,1.082347,0.771548,-1.433073,1.591338,0.238575,4.910314,0.0,0.102563,0.918310,0.122947
stroma_score[T.1],0.027688,1.028075,0.807588,-1.555156,1.610533,0.211156,5.005476,0.0,0.034285,0.972650,0.040008


In [11]:
runs=["All stages", "Stage i", "Stage ii", "Stage iii", "Stage iv"]
df["stroma_score"] = stroma_score

selected_columns = ["stroma_score" ,"cleanstage", "gender", "decades_to_birth" ]
df_mv = df.dropna(subset=["cleanstage", "decades_to_birth"])
y_mv = df_mv[["years_to_event","vital_status"]]

df_mv_i = df_mv
y_mv_i = y_mv
for i in range(len(runs)):
    if i != 0:
        y_mv_i = y_mv[df_mv["cleanstage"] == i]
        df_mv_i = df_mv[df_mv["cleanstage"] == i]
    mv_cox = CoxPHFitter().fit(pd.concat([df_mv_i[selected_columns], y_mv_i], axis=1), "years_to_event", "vital_status", formula = "stroma_score + cleanstage + C(gender) +decades_to_birth")
    # mv_cox.print_summary()

    stroma_HR = mv_cox.hazard_ratios_[3]
    lci = np.exp(mv_cox.confidence_intervals_["95% lower-bound"][3])
    hci = np.exp(mv_cox.confidence_intervals_["95% upper-bound"][3])
    p = mv_cox._compute_p_values()[3]
    print(f"___Deep Stroma Score Hazard Ratio, {runs[0]}___ \n Stroma HR: {stroma_HR}; \n CI: [{lci}, {hci}]; \n p: {p}")

___Deep Stroma Score Hazard Ratio, All stages___ 
 Stroma HR: 1.9854308859239034; 
 CI: [1.2654457286170446, 3.1150571799618465]; 
 p: 0.002841235882829334


c:\Users\amali\anaconda3\lib\site-packages\lifelines\utils\__init__.py:1102: ConvergenceWarning: Column(s) ['cleanstage'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
c:\Users\amali\anaconda3\lib\site-packages\lifelines\utils\__init__.py:800: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std


ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [87]:
runs=["All stages", "Stage i", "Stage ii", "Stage iii", "Stage iv"]
df["stroma_score"] = stroma_score

selected_columns = ["stroma_score" ,"cleanstage", "gender", "decades_to_birth" ]
df_mv = df.dropna(subset=["cleanstage", "decades_to_birth"])
y_mv = df_mv[["years_to_event","vital_status"]]

mv_cox = CoxPHFitter().fit(pd.concat([df_mv[selected_columns], y_mv], axis=1), "years_to_event", "vital_status", formula = "stroma_score + cleanstage + C(gender) +decades_to_birth")
# mv_cox.print_summary()

stroma_HR = mv_cox.hazard_ratios_[3]
lci = np.exp(mv_cox.confidence_intervals_["95% lower-bound"][3])
hci = np.exp(mv_cox.confidence_intervals_["95% upper-bound"][3])
p = mv_cox._compute_p_values()[3]
print(f"___Deep Stroma Score Hazard Ratio, {runs[0]}___ \n Stroma HR: {stroma_HR}; \n CI: [{lci}, {hci}]; \n p: {p}")

___Deep Stroma Score Hazard Ratio, All stages___ 
 Stroma HR: 1.9854308859239034; 
 CI: [1.2654457286170446, 3.1150571799618465]; 
 p: 0.002841235882829334
